# Project 5: Private Knowledge Chatbot (RAG)

## Executive Summary

This learning project builds a **private knowledge chatbot** using a classic **Retrieval-Augmented Generation (RAG)** pipeline.
You ingest a local knowledge base (PDFs), convert it into embeddings, store it in a vector database (Chroma), and then answer questions by retrieving the most relevant chunks and grounding the response on them.

### What this notebook does (end-to-end)
- Loads PDF documents from `knowledge-base/`
- Splits documents into overlapping text chunks (chunking)
- Creates vector embeddings for each chunk (OpenAI embeddings by default)
- Persists the embeddings into a local Chroma vector store (`vector_db/`)
- Builds a conversational retrieval chain with memory (LangChain)
- Prototypes an interactive chat UI with Gradio
- Visualizes the embedding space with t-SNE (2D and 3D) to sanity-check the vector store

### Why it matters (business framing)
Teams often have knowledge scattered across files and folders. RAG turns those documents into a searchable “semantic index”, enabling fast Q&A and better onboarding while keeping data local to the workspace.

### Key capabilities
- **Private knowledge ingestion:** local PDFs become searchable context
- **Semantic retrieval:** top-$k$ chunks are fetched per question
- **Conversational memory:** follow-up questions work better with chat history
- **Rapid prototyping:** a working chat UI in a few lines with Gradio
- **Observability:** lightweight embedding visualization for debugging

### Technical stack
- Python, LangChain, Chroma, OpenAI (embeddings + chat), scikit-learn (t-SNE), Plotly, Gradio

### Learning-oriented notes (what to pay attention to)
- Chunk size/overlap trade-offs (recall vs cost)
- Persistence and reset of the vector store (repeatable experiments)
- Retrieval parameters (`k`) and how they affect answer quality
- Limitations: this is a prototype (no formal evaluation, no strict citation formatting wired by default)


## Step-by-step roadmap (learning focus)

Run the notebook top-to-bottom. Each step below explains *why it exists*, what it produces, and what to watch for.

- **Step 0 — Imports & configuration:** bring in the libraries used for ingestion, vector storage, visualization, and UI.
- **Step 1 — Environment setup:** load API keys and pick the model + vector DB folder.
- **Step 2 — Document loading:** read PDFs from the local knowledge base folder.
- **Step 3 — Chunking:** split documents into chunks suitable for embedding + retrieval.
- **Step 4 — Vector store:** embed chunks and persist them to Chroma.
- **Step 5 — Sanity checks:** confirm vector counts/dimensions.
- **Step 6 — (Optional) Test questions:** keep a small question set for repeatable checks.
- **Step 7 — Visualization:** use t-SNE plots to understand the embedding space.
- **Step 8 — RAG chat chain:** connect retriever + LLM + memory.
- **Step 9 — Quick query:** validate the chain produces reasonable answers.
- **Step 10 — Gradio UI:** launch an interactive chat experience.

> This notebook intentionally keeps the code minimal and functional. Changes are focused on clarity, not new features.

## Step 0 — Imports

In a learning project, it helps to know *why each dependency exists*:
- **LangChain + Chroma**: ingestion, splitting, embeddings, retrieval, and the conversational chain
- **Plotly + scikit-learn (t-SNE)**: visualization to sanity-check the vector store
- **Gradio**: quick UI to interact with the chatbot

> If you get import errors, install missing packages in your notebook environment and re-run Step 0.

In [1]:
# Step 0) Imports (utilities + UI + visualization)

import os, shutil
import glob
from dotenv import load_dotenv
import gradio as gr
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Step 0b) Imports for LangChain + Chroma + embeddings + visualization

from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from sklearn.manifold import TSNE
from pathlib import Path
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings

/opt/conda/envs/LLM/lib/python3.11/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

## Step 1 — Configuration and credentials

This step defines (1) which chat model to use and (2) where the local vector database will be persisted.

**Inputs**
- `OPENAI_API_KEY` from your `.env` (or your environment variables)

**Outputs**
- A configured environment that downstream steps can use to embed text and call the chat model

**What to watch for**
- If `OPENAI_API_KEY` is not set, embedding or chat calls will fail later.

In [ ]:
# Step 1) Environment setup (.env + API keys)

# This repo commonly stores the .env file at /workspace/.env when running in containers.
# We keep a safe fallback to the local .env (current working directory).
dotenv_path = os.getenv("DOTENV_PATH", "/workspace/.env")
load_dotenv(dotenv_path=dotenv_path, override=True)
load_dotenv(override=True)

# Ensure the OpenAI SDK sees the key (set it in your .env for real usage).
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your-key-if-not-using-env")

## Step 2 — Load documents (private knowledge base)

We load PDFs from the local `knowledge-base/` folder. Each PDF is treated as a set of pages. In RAG, pages (or page chunks) become the raw material for retrieval.

**Output**
- A list of `Document` objects (typically one per page), including metadata like source file name and page number.

In [ ]:
# Step 2) Load PDFs (from knowledge-base/)

KB_DIR = Path("knowledge-base").resolve()
print("KB_DIR:", KB_DIR)

loader = DirectoryLoader(
    str(KB_DIR),
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
)

documents = loader.load()
print("Loaded pages:", len(documents))
print("Example metadata:", documents[0].metadata if documents else None)

KB_DIR: /workspace/giova/llms-engineering-main/knowledge-base


100%|██████████| 1/1 [00:01<00:00,  1.08s/it]

Docs (páginas) cargadas: 56
Ejemplo metadata: {'producer': 'Acrobat Distiller 9.4.0 (Windows)', 'creator': 'Epic Editor v. 5.0', 'creationdate': '2011-12-14T14:40:39+00:00', 'author': 'SPSS Inc.', 'moddate': '2011-12-14T14:40:39+00:00', 'title': 'Manual CRISP-DM de IBM SPSS Modeler', 'source': '/workspace/giova/llms-engineering-main/knowledge-base/crisdm.pdf', 'total_pages': 56, 'page': 0, 'page_label': '1'}


## Step 3 — Chunking strategy

LLMs and embedding models work best with smaller passages. Chunking splits pages into overlapping windows so retrieval can surface the most relevant context.

**Key idea**: overlap helps preserve context across chunk boundaries, improving recall for questions that span sentences/sections.

In [ ]:
# Step 3) Chunking (split pages into overlapping text chunks)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print("Chunks:", len(chunks))
print("Chunk snippet:", chunks[0].page_content[:200] if chunks else None)

Chunks: 55
Snippet chunk: i
Manual CRISP-DM de IBM SPSS
Modeler


## Step 4 — Embeddings + vector store (Chroma)

In this step we convert each chunk into a numeric embedding vector and persist everything in Chroma. This is what enables semantic search: questions and chunks live in the same vector space.

**Outputs**
- A persisted Chroma vector store in `vector_db/`
- An internal collection with `count()` vectors (one per chunk)

In [ ]:
# Step 4) Build the vector store (embeddings + persistence)
# - Each chunk becomes an embedding vector
# - Chroma persists the collection locally (SQLite under the hood)

embeddings = OpenAIEmbeddings()

# Optional: use free Hugging Face sentence-transformer embeddings instead of OpenAI
# Example:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Reset the persisted collection to keep experiments repeatable
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create and persist vector store
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 55 documents


In [ ]:
# Step 5) Sanity check: vector count + embedding dimensions

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 55 vectors with 1,536 dimensions in the vector store


## Step 5 — Quick sanity checks

Before building the chat layer, confirm the vector store looks reasonable: number of vectors and embedding dimensionality.

- Too few vectors usually means document loading/chunking failed.
- Unexpected dimensions can indicate a different embedding model than you intended.

In [ ]:
# Step 6) Test question set (repeatable checks + grounding)

test_questions = [
    "What does CRISP-DM mean and what is it used for?",
    "How many phases does CRISP-DM have?",
    "List the 6 CRISP-DM phases in order.",
    "What happens in the Business Understanding phase?",
    "What happens in the Data Understanding phase?",
    "What typical tasks are mentioned in Data Preparation?",
    "Why is CRISP-DM considered an iterative process?",
    "Give me a summary of 'Business Understanding' and cite pages.",
    "Explain what the document says about risks/contingencies and cite pages.",
    "If my goal is to reduce customer churn, which phase should I do first and why?",
    "Rewrite that plan as 6 bullets aligned to the CRISP-DM phases.",
]

## Step 6 — A tiny evaluation loop (optional)

This list is a lightweight way to keep your checks repeatable while iterating on chunking and retrieval settings.

In a production system you would add a real evaluation harness (golden answers, citations, and scoring). For this learning project, a small question set is enough to spot obvious regressions.

## Step 7 — Visualizing the vector store (debugging intuition)

This is not required for RAG to work, but it is extremely helpful when you are learning.
Visualization helps you build intuition about:
- Whether embeddings look “clustered” vs random noise
- Whether document types (if present in metadata) separate in space
- Whether there are obvious outliers (often caused by very short/empty chunks)

We use t-SNE (2D and 3D) as a quick exploratory tool.

In [ ]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']

# Safe doc_type extraction (fallback to 'unknown')
doc_types = [m.get("doc_type", "unknown") for m in metadatas]

# Assign one color per distinct type (without assuming known names)
unique_types = sorted(set(doc_types))
palette = ["blue", "green", "red", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

type_to_color = {t: palette[i % len(palette)] for i, t in enumerate(unique_types)}
colors = [type_to_color[t] for t in doc_types]


In [ ]:
pio.renderers.default = "iframe"  # most robust option for remote Jupyter/JupyterLab

In [ ]:
# --- Ensure vectors is a 2D NumPy array ---
vectors = np.array(vectors)
assert vectors.ndim == 2, f"vectors must be 2D, got shape: {vectors.shape}"

n = vectors.shape[0]
if n < 2:
    raise ValueError(f"Not enough embeddings to visualize (n={n}).")

# --- t-SNE: adapt perplexity to the number of points ---
# Rule of thumb: perplexity < n, typically <= (n-1)/3
perplexity = min(30, max(2, (n - 1) // 3))
tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity, init="pca", learning_rate="auto")
reduced_vectors = tsne.fit_transform(vectors)

# --- Hover text for PDFs: source + page + type + snippet ---
def safe_snippet(text, k=200):
    if not text:
        return ""
    text = text.replace("\n", " ").strip()
    return (text[:k] + "...") if len(text) > k else text

hover_text = []
for md, t, d in zip(metadatas, doc_types, documents):
    src = os.path.basename(md.get("source", "unknown"))
    page = md.get("page", md.get("page_number", ""))
    page_str = f"{page}" if page != "" else "?"
    hover_text.append(
        f"Type: {t}"
        f"<br>Source: {src}"
        f"<br>Page: {page_str}"
        f"<br>Text: {safe_snippet(d)}"
    )

# --- Plot 2D ---
fig = go.Figure(
    data=[go.Scatter(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        mode="markers",
        marker=dict(size=6, color=colors, opacity=0.85),
        text=hover_text,
        hoverinfo="text"
    )]
 )

fig.update_layout(
    title=f"2D Chroma Vector Store (t-SNE, n={n}, perplexity={perplexity})",
    xaxis_title="t-SNE x",
    yaxis_title="t-SNE y",
    width=900,
    height=650,
    margin=dict(r=20, b=20, l=20, t=60)
 )

fig.show()


In [ ]:
pio.renderers.default = "iframe"  # most robust option for remote Jupyter/JupyterLab

vectors = np.array(vectors)
n = vectors.shape[0]
assert n == 55, f"Expected 55 vectors, got {n}"

tsne = TSNE(
    n_components=3,
    random_state=42,
    perplexity=15,      # tuned for n=55
    init="pca",
    learning_rate="auto",
)
reduced_vectors = tsne.fit_transform(vectors)

fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text',
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='t-SNE x', yaxis_title='t-SNE y', zaxis_title='t-SNE z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Step 8 — Build the RAG chat chain (LangChain)

Now we connect three pieces:
- **Retriever** (Chroma): finds the top-$k$ most relevant chunks
- **LLM** (Chat model): writes the final answer using retrieved context
- **Memory**: keeps chat history so follow-ups make sense

This is the core “knowledge worker” behavior: retrieve → ground → respond.

In [ ]:
# Step 8) Create the conversational retrieval chain (LLM + retriever + memory)
llm = ChatOpenAI(model=MODEL, temperature=0.7)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Alternative (local): if you'd like to use Ollama, you can switch the backend like this:
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# Conversation memory stores the chat history (useful for follow-ups)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Build the RAG chain: retrieval + chat model + memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/tmp/ipykernel_3251/1344145089.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [ ]:
# Quick smoke test: ask one question and inspect the answer text

query = "What happens in the Business Understanding phase?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

En la fase de Comprensión del negocio, se realizan varias actividades clave para establecer un marco claro para el proyecto de minería de datos. Estas actividades incluyen:

1. **Definir objetivos comerciales**: Se identifican y documentan los objetivos específicos que la organización espera alcanzar con la minería de datos. Esto ayuda a alinear los esfuerzos del proyecto con las metas empresariales.

2. **Valorar la situación actual**: Se evalúa la situación comercial existente, incluyendo la disponibilidad de datos, recursos humanos, problemas existentes y factores de riesgo. Esto permite tener una visión clara de los recursos y limitaciones del proyecto.

3. **Identificar áreas problemáticas**: Se determina el área específica que necesita atención, como marketing, atención al cliente o desarrollo comercial, y se describe el problema general que se busca resolver.

4. **Compilación de información**: Se recopila información sobre la empresa, como la estructura organizativa, los recurs

In [ ]:
# Step 9) Reset memory (fresh chat session)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Rebuild the chain with fresh memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Step 10 — Gradio UI (interactive demo)

A chat UI makes it easy to test the system like an end user would: ask questions, refine prompts, and quickly validate improvements.

This is a prototype UI (learning-first): no auth, no rate limiting, and minimal logging.

In [ ]:
# Step 10) Wrap the chain into a simple function for Gradio

def chat(question, history):
    # Note: Gradio provides `history`, but the LangChain memory inside `conversation_chain` is what we use here.
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a32eeddc47c7c6bf9b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
